In [ ]:
import intake
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd

import cartopy.crs as ccrs

import holoviews as hv
import hvplot.xarray
import hvplot.pandas

from geoviews.tile_sources import EsriImagery

hv.extension('bokeh')

In [ ]:
mtf_source = intake.open_zarr('data/mean_temp.zarr')

In [ ]:
mtf = mtf_source.to_dask()
mtf.persist()

In [ ]:
from holoviews import opts
import panel as pn
import colorcet as cc

special_cmap = cc.fire[::-1][90:]

In [ ]:
p = (mtf
    .hvplot(x='x', y='y', geo=True, height=500, width=550, cmap='rainbow', alpha=.5, legend=False)
    .relabel('Mean Surface Temp (F)'))

p * EsriImagery

In [ ]:
clipped  = p.opts(opts.Image(clipping_colors={'min': 'transparent', 'max': 'transparent'}, 
                             cmap=special_cmap),
                  clone=True)
clipped.redim.range(value=(90,120))

In [ ]:
start, end = int(mtf.temperature.min())-1, int(mtf.temperature.max()+1)
slider = pn.widgets.RangeSlider(start=start, end=end, value=(start, end), 
                                step=1, bar_color='#ff0000')

slider.jslink(clipped, code={'value': """
color_mapper.low = source.value[0];
color_mapper.high = source.value[1];
console.log(color_mapper);
"""})

pn.Row(slider, clipped * EsriImagery)

In [ ]:
import geoviews as gv
from geoviews.tile_sources import tile_sources

In [ ]:
tile_source = gv.tile_sources.WMTS(EsriImagery.data)
selector = pn.widgets.Select(name='basemap', value=EsriImagery.data, options={k: v.data for k, v in tile_sources.items()})
selector.jslink(tile_source, value='glyph.url')

pn.Row(selector, clipped * tile_source)

In [ ]:
from holoviews.streams import Params

In [ ]:
start, end = int(mtf.temperature.min())-1, int(mtf.temperature.max()+1)
slider = pn.widgets.RangeSlider(start=start, end=end, value=(start, end), 
                                step=1, bar_color='#ff0000')

slider.jslink(clipped, code={'value': """
color_mapper.low = source.value[0];
color_mapper.high = source.value[1];
"""})

selector = pn.widgets.Select(name='basemap', value=EsriImagery, options=tile_sources)
selector.param.watch(set_tile_source, 'value')

def set_tile_source(value):
    return value
    
dmap = hv.DynamicMap(set_tile_source, streams=[Params(selector, ['value'])])

pn.Row(
    pn.Column(
        slider, 
        selector),
    clipped * dmap)

> To Do: Add an alpha slider, toggle off basemap, toggle off trees, change dynspread of trees

### Pure JS - do not recommend

In [ ]:
tile_source = gv.tile_sources.WMTS(EsriImagery.data)
selector = pn.widgets.Select(name='basemap', value=EsriImagery.data, options=[ts.data for ts in tile_sources.values()])
selector.jslink(tile_source, value='glyph.url')

pn.Row(
    pn.Column(selector, slider), 
    clipped * tile_source)

### Other ideas

In [ ]:
hv_obj = hv.Image(np.random.rand(10, 10))

low_cpicker = pn.widgets.ColorPicker()
high_cpicker = pn.widgets.ColorPicker(value='#ffffff')
slider = pn.widgets.RangeSlider(step=0.01)

slider.jslink(hv_obj, code={'value': """
color_mapper.low = source.value[0];
color_mapper.high = source.value[1];
"""})

low_cpicker.jslink(hv_obj, value='color_mapper.low_color')
high_cpicker.jslink(hv_obj, value='color_mapper.high_color')

pn.Row(hv_obj, pn.Column(slider, low_cpicker, high_cpicker))

## Adding in the Street Tree data

OpenDataPhilly released an inventory of all the street trees in the city. Street trees are trees that are planted along streets, not those in parks and private property. Using `datashader` we can easily plot these 100,000 points.


It is hypothesized that where there are more trees the land surface temperature will be less extreme. To explore this, we will overlay street trees with the thresholded land surface temperature:

In [ ]:
url = 'http://data.phl.opendata.arcgis.com/datasets/957f032f9c874327a1ad800abd887d17_0.geojson'
trees = gpd.read_file(url)

In [ ]:
tree_p = trees.hvplot(geo=True, datashade=True, height=500, width=500, 
                      legend=False, dynspread=30).relabel('Street Tree Density')

thresholded_temp_p.options(alpha=.5) * tree_p.options(alpha=.5)

In [ ]:
tree_p = trees.hvplot(geo=True, datashade=True, height=500, width=500, 
                      legend=False, dynspread=30).relabel('Street Tree Density')